### Importing Dependencies

In [2]:
import pandas as pd
import os
import requests
import json
# from config import password
from sqlalchemy import create_engine

### Creating a function to import csv data from multiple countries

In [3]:
def import_func(country_code):
    
    #Creating Path
    
    path = os.path.join('data',f'{country_code}_youtube_trending_data.csv')
    
    
    #Storing dataframe to df
    
    df=pd.read_csv(path)
    
    #Remving unwanted columns below
    
    df=df[['video_id','title','publishedAt','channelTitle','categoryId','trending_date','view_count','likes','dislikes','comment_count','thumbnail_link']]
    
    
    #Changing object types to date types for two columns
    
    df['publishedAt']=pd.to_datetime(df['publishedAt'])
    df['trending_date']=pd.to_datetime(df['trending_date'])
    
    
    #Removing time stamp from date
    
    df['publishedAt']=df['publishedAt'].dt.date 
    df['trending_date']=df['trending_date'].dt.date
    
    
    with open(f'data/{country_code}_category_id.json', 'r') as read_file:
        category_ids = json.load(read_file)

        df=df.astype({'categoryId': 'str'})
        
    for index,row in df.iterrows():
    
        for entry in category_ids["items"]:

            if row["categoryId"]==entry["id"]:
                df.at[index,"categoryId"]=entry["snippet"]["title"]
    
    #Adding country Code as column
    df['country']=f'{country_code}'
    col_name='country'
    
    
    #Moving country code to first column
    first_col = df.pop(col_name)
    df.insert(0,col_name,first_col)
    
    return df

### Creating all dataframes using import function

In [4]:
#USA Dataframe
df_us=import_func('US')

#Brasil Dataframe
df_br=import_func('BR')

#Canada Dataframe
df_ca=import_func('CA')

#Mexico Dataframe
df_mx=import_func('MX')

### Combining all data to one dataframe

In [10]:
#Creating a varible to add all dfs
country_df=[df_us,df_br,df_ca,df_mx]

df_main = pd.concat(country_df)

,country,video_id,title,publishedAt,channelTitle,categoryId,trending_date,view_count,likes,dislikes,comment_count,thumbnail_link
0,US,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11,Brawadis,People & Blogs,2020-08-12,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg
1,US,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11,Apex Legends,Gaming,2020-08-12,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg
2,US,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11,jacksepticeye,Entertainment,2020-08-12,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg
3,US,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11,XXL,Music,2020-08-12,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg
4,US,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11,Mr. Kate,Howto & Style,2020-08-12,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg


### Creating SQL engine to connnect to Postgres

In [12]:
# #Connect to Database

# pg_user = 'postgres'
# pg_password=password
# db_name = 'youtube_database'
# connection_string = f'{pg_user}:{password}@localhost:5432/{db_name}'
# engine=create_engine(f'postgresql://{connection_string}')

#Connect to Database - Alternative
dbuser = 'postgres'
dbpassword =password
dbhost = 'localhost'
dbport = '5432'
dbname= 'youtube_database'
connection_string2 = f'{dbuser}:{dbpassword}@database-1.cvmfiiilpm7y.us-east-1.rds.amazonaws.com:{dbport}/{dbname}'
engine=create_engine(f'postgresql://{connection_string2}')

In [13]:
engine.table_names()

[]

In [14]:
df_main.to_sql(name='youtube_table',con=engine,if_exists='append',index=False)